# Runtime Test

For conver yolov5 model to trt plan (**yolov5s-2021-07-28.engine**) and build **libmyplugins.so** for your system see instruction: https://github.com/wang-xinyu/tensorrtx/tree/master/yolov5#how-to-run-yolov5s-as-example

Based on this instruction, we wrote a small shell script.

```bash
cd ../inference/convertors/yolo2tensorrt/bin/
./yolov5_tensorrt.sh
```
Before runing this script, please download install additionaly **opencv-contrib** and **pycuda**

   * Copy the resulting **libmyplugins.so** to the folder from which you will run this notebook
   * Replace the data model **data/models/Detector/yolov5engine/yolov5s-2021-07-28.engine** with the one you just got from running the script

Convert options and ocr
* options (inference/convertors/options2tensorrt/convert_numberplate_options_to_onnx.py)
* ocrs (inference/convertors/ocr2tensorrt/convert_ocr_to_onnx.py)

In [ ]:
import warnings
import os
from glob import glob

from _paths import current_dir
from nomeroff_net.pipelines.number_plate_detection_and_reading_trt_runtime\
    import NumberPlateDetectionAndReadingTrtRuntime

In [ ]:
warnings.filterwarnings("ignore")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
number_plate_detection_and_reading_trt_runtime = NumberPlateDetectionAndReadingTrtRuntime(
    "number_plate_detection_and_reading_runtime", 
    image_loader="opencv" # Try 'turbo' for faster performance.
)

In [ ]:
num_run = 1
batch_size = 1
num_workers = 1
images = glob(os.path.join(current_dir, './images/*'))

number_plate_detection_and_reading_trt_runtime.clear_stat()
outputs = number_plate_detection_and_reading_trt_runtime(
    images, 
    batch_size=batch_size,
    num_workers=num_workers)
number_plate_detection_and_reading_trt_runtime.clear_stat()

for i in range(num_run):
    print(f"pass {i}")
    outputs = number_plate_detection_and_reading_trt_runtime(
        images, 
        batch_size=batch_size,
        num_workers=num_workers)

In [ ]:
timer_stat = number_plate_detection_and_reading_trt_runtime.get_timer_stat(len(images)*num_run)

### Jetson Xavier Tensorrt

In [ ]:
print(f"Processed {len(images)} photos")
print(f"One photo process {timer_stat['NumberPlateDetectionAndReadingRuntime.call']} seconds")
print()
print(f"detect_bbox_time_all {timer_stat['NumberPlateLocalization.call']} per one photo")
print(f"craft_time_all {timer_stat['NumberPlateKeyPointsDetection.call']} per one photo")
print(f"classification_time_all {timer_stat['NumberPlateClassification.call']} per one photo")
print(f"ocr_time_all {timer_stat['NumberPlateTextReading.call']} per one photo")

### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz + RTX 3090 Tensorrt in Docker